In [6]:
%matplotlib inline

In [7]:
from __future__ import print_function
from __future__ import absolute_import

import warnings
import numpy as np
import h5py
from keras.preprocessing import image
from sklearn.model_selection import train_test_split
from keras.models import Model
from keras import layers
from keras.layers import Dense
from keras.layers import Input
from keras.layers import BatchNormalization
from keras.layers import Activation
from keras.layers import Conv2D
from keras.layers import SeparableConv2D
from keras.layers import MaxPooling2D
from keras.layers import GlobalAveragePooling2D
from keras.layers import GlobalMaxPooling2D
from keras.engine.topology import get_source_inputs
from keras.utils.data_utils import get_file
from keras import backend as K
from keras.applications.imagenet_utils import decode_predictions
from keras.applications.imagenet_utils import _obtain_input_shape

In [8]:
import cv2 as cv2

In [27]:
comb_data = h5py.File('/local/viraj/courses/ME592x/assn3/Aditya_data/combustion_img_13.mat','r')
X = comb_data['train_set_x'][()]
y = comb_data['train_set_y'][()]

In [28]:
X_trf, X_tsf,y_trf, y_tsf = train_test_split(X.T, y, test_size=0.4)

In [29]:
#Resizing the image to lower resolution in (width, height): CHANGE THIS PARAMETER FOR DIFFERENT RESOLUTIONS
width = 100
height = 100
num_features = width*height

In [30]:
a=np.shape(X_trf)[0]/10
X_train_final = np.zeros([a,num_features])
for i in range(0,a):
    temp_image = X_trf[i,:] 
    temp_image = np.reshape(temp_image,[250,100])
    temp_image = temp_image.T
    temp_image = cv2.resize(temp_image, dsize=((width,height)))
    temp_image = np.reshape(temp_image,[num_features,])
    X_train_final[i,:] = temp_image
    
b = np.shape(X_tsf)[0]/10
X_test_final = np.zeros([b,num_features])
for i in range(0,b):
    temp_image = X_tsf[i,:] 
    temp_image = np.reshape(temp_image,[250,100])
    temp_image = temp_image.T
    temp_image = cv2.resize(temp_image, dsize=(width,height))
    temp_image = np.reshape(temp_image,[num_features,])
    X_test_final[i,:] = temp_image

In [31]:
X_train_final = X_train_final.reshape([a,height,width,1])
X_test_final = X_test_final.reshape([b,height,width,1])

In [21]:
#X_train_final = np.concatenate((X_train_final,X_train_final,X_train_final), axis = 3)
#X_test_final = np.concatenate((X_test_final,X_test_final,X_test_final), axis = 3)

In [32]:
from keras.utils import to_categorical
y_trf_oh = to_categorical(y_trf)
y_tsf_oh = to_categorical(y_tsf)

In [46]:
def SmallXception(input_shape=(100, 100, 1),
                  classes=2):
    img_input = Input(shape=input_shape)

    x = Conv2D(32, (3, 3), strides=(2, 2), use_bias=False)(img_input)
    #x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D((3, 3), strides=(2, 2))(x)

    x = SeparableConv2D(128, (3, 3), use_bias=False)(x)
    #x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = SeparableConv2D(128, (3, 3), use_bias=False)(x)
    #x = BatchNormalization()(x)

    x = MaxPooling2D((3, 3), strides=(2, 2))(x)

    x = SeparableConv2D(256, (3, 3), use_bias=False)(x)
    #x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = SeparableConv2D(256, (3, 3), use_bias=False)(x)
    #x = BatchNormalization()(x)

    x = MaxPooling2D((3, 3), strides=(2, 2))(x)

    #x = SeparableConv2D(512, (3, 3), use_bias=False)(x)
    #x = BatchNormalization()(x)
    #x = Activation('relu')(x)
    #x = SeparableConv2D(512, (3, 3), use_bias=False)(x)
    #x = BatchNormalization()(x)

    x = GlobalAveragePooling2D(name='avg_pool')(x)
    x = Dense(classes, activation='softmax', name='predictions')(x)

    # Create model.
    model = Model(img_input, x)
    return model

In [47]:
if __name__ == '__main__':
    model = SmallXception()
    model.summary()
    
    print('Input image shape:', X_train_final.shape)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.fit(X_train_final, y_trf_oh[0:a], epochs=1, batch_size=32)
   
    
    scores = model.evaluate(X_test_final, y_tsf_oh[0:b], verbose=0)
    print("Accuracy: %.2f%%" % (scores[1]*100))

ValueError: Negative dimension size caused by subtracting 3 from 2 for 'separable_conv2d_32/separable_conv2d/depthwise' (op: 'DepthwiseConv2dNative') with input shapes: [?,2,2,256], [3,3,256,1].

In [37]:
scores = model.evaluate(X_train_final, y_trf_oh[0:a], verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 54.69%


In [40]:
if __name__ == '__main__':
    model = Xception(include_top=True, weights=None, classes=2)

    #img_path = 'baboon.png'
    #img = image.load_img(img_path, target_size=(299, 299))
    #x = image.img_to_array(img)
    #x = np.expand_dims(x, axis=0)
    #x = preprocess_input(x)
    
    print('Input image shape:', X_train_final.shape)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.fit(X_train_final, y_trf_oh[0:a], epochs=1, batch_size=16)
   
    
    scores = model.evaluate(X_test_final, y_tsf_oh[0:b], verbose=0)
    print("Accuracy: %.2f%%" % (scores[1]*100))

Input image shape: (4320, 299, 299, 3)
Epoch 1/1
 608/4320 [===>..........................] - ETA: 2043s - loss: 0.2275 - acc: 0.9211

KeyboardInterrupt: 

In [15]:

TF_WEIGHTS_PATH = 'https://github.com/fchollet/deep-learning-models/releases/download/v0.4/xception_weights_tf_dim_ordering_tf_kernels.h5'
TF_WEIGHTS_PATH_NO_TOP = 'https://github.com/fchollet/deep-learning-models/releases/download/v0.4/xception_weights_tf_dim_ordering_tf_kernels_notop.h5'